In [1]:
import serial
import time
import pyvisa 
import numpy as np
from IPython.display import Image, display

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = 'COM3'

MHZ = 1000000

In [2]:
# helper for sending serial commands
# expects a string with the command (they dont have to be escaped with \r or \n at the end)
# if echo is set to false, that means not to worry about the response from the
# pico it signifigantly reduces communication time when sending many
# instructions, but you lose out on the debugging info from the pico
# def send(command: str, echo = True) -> str:
#     # pico is expecting a newline to end every command
#     if command[-1] != '\n':
#         command += '\n'

#     resp = ''
#     conn = None
#     try:
#         conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)
#         conn.write(command.encode())
#         if echo:
#             resp = conn.readlines()
#             resp = "".join([s.decode() for s in resp])

#     except Exception as e:
#         print("Encountered Error: ", e)

#     finally:
#         conn.close()

#     return resp

conn = None
conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)

def assert_OK():
    resp = conn.readline().decode().strip()
    assert resp == 'ok', 'Expected "ok", received "%s"' % resp

def sendb(instructions: np.ndarray, echo = True) -> str:

    resp = ''

    try:

        ## tell setb how many instructions to expect 
        conn.write(b'setb 0 %d\n' % len(instructions))
        # assert_OK(), 'something wrong with instructions'

        response = conn.readline().decode()
        if not response.startswith('ready'):
            response += ''.join([r.decode() for r in conn.readlines()])
            raise Exception(f'setb command failed, got response {repr(response)}')
        # print(f'table ready, got response: {response}')

        conn.write(instructions.tobytes())
        assert_OK(), 'table not written correctly'
        print('table written')
        
        conn.write(b'set 4 %d\n' % len(instructions))
        assert_OK(), 'table not stopped correctly'
        print('table finished')

        # conn.write(b'start\n')
        # print('start sent')

    except Exception as e:
        raise e

    # finally:
    #     conn.close()

    return resp

In [3]:
## slightly different logic than the ad9959.c funcs due to python vs c
def get_ftw(freq_out: float, freq_sys: float = 500 * MHZ):

    ftw = (freq_out / freq_sys) * 2**32
    
    return(int(ftw))

def get_pow(phase: float):
    
    pow = round(phase / 360 * (2**14 - 1))

    return pow

def get_asf(amp: float):
    
    asf = round(amp * 1024)

    return asf

## Single Stepping Mode

In [4]:
dt = np.dtype([('frequency', np.uint32), ('amplitude', np.uint16), ('phase', np.uint16), ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 1

example_instructions = np.array([
                                (f2, 600, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 800, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 800, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 800, p1, t)
                                 ]
                                 , dtype = dt)

conn.write(b'reset\n')
assert_OK()
conn.write(b'mode 0 1\n')
assert_OK()
conn.write(b'setchannels 1\n')
assert_OK()

In [5]:
sendb(example_instructions)

table written
table finished


''

In [6]:
conn.write(b'start\n')

6

In [7]:
assert_OK()

## Frequency Sweeps

Frequency Sweeps (mode 2) with timing: `<start frequency:int 32> <stop frequency:int 32> <delta:int 32> <rate:int 8> <time:int 32>`. Total of 17 bytes per channel per instruction.

In [8]:
dt = np.dtype([('start frequency', np.uint32), 
               ('stop frequency', np.uint32), 
               ('delta', np.uint32), 
               ('rate', np.uint8), 
               ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

d = 100
r = 1
t = 1

example_instructions = np.array([
                                (f2, f1, d, r, t),
                                (f1, f3, d, r, t),
                                (f3, f2, d, r, t),
                                (f2, f1, d, r, t),
                                (f1, f3, d, r, t)
                                 ]
                                 , dtype = dt)

conn.write(b'reset\n')
assert_OK()
conn.write(b'mode 2 1\n')
assert_OK()
conn.write(b'setchannels 1\n')
assert_OK()



In [9]:
sendb(example_instructions)

table written
table finished


''

In [10]:
conn.write(b'start\n')


6

In [11]:
assert_OK()
